In [1]:
import pandas as pd
import numpy as np
import os
import sys
from  networkx import bipartite
sys.path.append('./../..')
sys.path.append('./..')
import glob
from tqdm import tqdm
import joblib
from joblib import Parallel, delayed
from pandarallel import pandarallel
from gensim.models import KeyedVectors
import multiprocessing as mp
pandarallel.initialize()
import re
from IPython.core.display import display, HTML
import yaml
from sklearn.preprocessing import LabelEncoder
import sys
sys.path.append('./..')
sys.path.append('./../..')
from common_utils import utils
from collections import Counter
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool
import time
import itertools
import matplotlib.pyplot as plt
import networkx as nx
id_col = 'PanjivaRecordID'
import networkx as nx
import operator
import collections
import argparse
from pathlib import Path
from networkx.algorithms import community
import json
# -------------------------------------------------
config_ANOM_PERC = 1

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
config_SIMTYPE = 4  # This means how many similar "types" of a transaction should be generated should be there

In [3]:
CONFIG = None
DIR_LOC = None
CONFIG = None
CONFIG_FILE = 'config.yaml'
save_dir = None
id_col = 'PanjivaRecordID'
use_cols = None
freq_bound = None
attribute_columns = None
DIR = None
CUT_OFF = None
company_attributes = ['ConsigneePanjivaID','ShipperPanjivaID']

In [4]:
# -------------------
# To find Betwenness Centrality
# -------------------

def chunks(l, n):
    """Divide a list of nodes `l` in `n` chunks"""
    l_c = iter(l)
    while 1:
        x = tuple(itertools.islice(l_c, n))
        if not x:
            return
        yield x


def betweenness_centrality_parallel(G, processes=None):
    """Parallel betweenness centrality  function"""
    p = Pool(processes=processes)
    node_divisor = len(p._pool) * 4
    node_chunks = list(chunks(G.nodes(), int(G.order() / node_divisor)))
    num_chunks = len(node_chunks)
    bt_sc = p.starmap(
        nx.betweenness_centrality_subset,
        zip(
            [G] * num_chunks,
            node_chunks,
            [list(G)] * num_chunks,
            [True] * num_chunks,
            [None] * num_chunks,
        ),
    )

    # Reduce the partial solutions
    bt_c = bt_sc[0]
    for bt in bt_sc[1:]:
        for n in bt:
            bt_c[n] += bt[n]
    return bt_c

In [5]:
def set_up_config(_DIR=None):
    global DIR
    global CONFIG
    global CONFIG_FILE
    global use_cols
    global num_neg_samples
    global DATA_SOURCE
    global DIR_LOC
    global save_dir
    global id_col
    global attribute_columns

    DATA_SOURCE = './../generated_data_v1/'
    with open(CONFIG_FILE) as f:
        CONFIG = yaml.safe_load(f)

    if _DIR is not None:
        DIR = _DIR
        CONFIG['DIR'] = _DIR
    else:
        DIR = CONFIG['DIR']

    DIR_LOC = re.sub('[0-9]', '', DIR)
    DATA_SOURCE = os.path.join(DATA_SOURCE, DIR)
    save_dir = '__2'
    save_dir = os.path.join(
        DATA_SOURCE,
        save_dir
    )
    

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    use_cols = CONFIG[DIR]['use_cols']
    _cols = list(use_cols)
    _cols.remove(id_col)
    attribute_columns = list(sorted(_cols))
    return

In [6]:
DIR = 'us_import2'
set_up_config(DIR)

Anomaly_Output_path = './../generated_data_v1/generated_anomalies/{}'.format(DIR)
path = Path(Anomaly_Output_path)
path.mkdir(exist_ok=True, parents=True)

In [7]:
df_train = pd.read_csv(
    os.path.join(DATA_SOURCE, 'train_data.csv'),
    low_memory=False,
    index_col=None
)

df_test = pd.read_csv(
    os.path.join(DATA_SOURCE, 'test_data.csv'),
    low_memory=False,
    index_col=None
)

In [8]:
def check_degree_distribution(graph_obj):
    degree_sequence = sorted([d for n, d in graph_obj.degree()], reverse=True)
    try:
        plt.title('Boxplot of degree')
        plt.boxplot(degree_sequence)
        plt.show()
    except:
        pass
    degreeCount = collections.Counter(degree_sequence)
    deg, cnt = zip(*degreeCount.items())
    try:
        fig, ax = plt.subplots()
        plt.bar(deg, cnt, width=0.80, color="b")
        plt.title("Degree Histogram")
        plt.ylabel("Count")
        plt.xlabel("Degree")
        plt.show()
    except:
        pass
   

In [9]:
def get_domain_dims():
    global DATA_SOURCE
    
    dom_dims_path = os.path.join(DATA_SOURCE, 'domain_dims.pkl')
    with open(dom_dims_path,'rb') as fh:
        domain_dims =  pickle.load(fh)
    return domain_dims

In [10]:
target_df = utils.convert_to_serializedID_format( target_df = df_train, DIR=DIR, data_source_loc=None,REFRESH=False)

100%|██████████| 8/8 [00:17<00:00,  2.22s/it]


In [11]:
main_df = target_df.copy()

In [12]:

def build_complete_graph(df):
    from itertools import combinations
    domain_dims = get_domain_dims()
    complete_G =nx.Graph()
    edges = [
        ['Carrier','ConsigneePanjivaID'],
        ['Carrier','HSCode'],
        ['Carrier','PortOfLading'],
        ['Carrier','PortOfUnlading'],
        ['Carrier','ShipperPanjivaID'],
        ['ConsigneePanjivaID','HSCode'],
        ['ConsigneePanjivaID','PortOfUnlading'],
        ['ConsigneePanjivaID','ShipmentDestination'],
        ['ConsigneePanjivaID','ShipperPanjivaID'],
        ['HSCode','ShipmentDestination'],
        ['HSCode','ShipmentOrigin'],
        ['HSCode','ShipperPanjivaID'],
        ['PortOfLading','ShipmentOrigin'],
        ['PortOfLading','ShipperPanjivaID'],
        ['PortOfUnlading','ShipmentDestination'],
        ['ConsigneePanjivaID','ShipperPanjivaID']
    ]
    
    for i,j in edges:
        __tmp__ = df[[i,j]]
        __tmp__ = __tmp__.drop_duplicates()
        _i = __tmp__[i].values.tolist()
        _j = __tmp__[j].values.tolist()
        n1_n2 = [ ( a,b)for a,b in zip(_i,_j)]
        complete_G.add_edges_from(n1_n2)
    return complete_G
        
complete_G = build_complete_graph(target_df)    

In [13]:
df_test = utils.convert_to_serializedID_format( target_df = df_test, DIR=DIR, data_source_loc=None,REFRESH=False)

100%|██████████| 8/8 [00:18<00:00,  2.27s/it]


In [14]:
domain_dims = get_domain_dims()
entity_type_list = {
    d : list(set(target_df[d])) for d in domain_dims.keys()
}

In [15]:
def get_largest_component_subgraph(graph_obj):
    component_id = 0
    components = {}
    component_size_dict = {}

    for c in nx.connected_components(graph_obj):
        components[component_id] = c
        component_size_dict[component_id] = len(c) 
        component_id += 1
        
    component_size_dict = sorted(component_size_dict.items(), key=operator.itemgetter(1), reverse=True)
    # Get the largest connected component
    max_component = components[component_size_dict[0][0]]
    subgraph = graph_obj.subgraph(max_component)
    return subgraph , component_size_dict

In [16]:
# --------------------------------------
# Used to get approximate candidates 
# --------------------------------------
def get_n2v_model(G):
    global DIR
    fname = DIR +'_node2vec'
    from nodevectors import Node2Vec
    if not os.path.exists(fname + '.zip'):
        n2v = Node2Vec()
        n2v.fit(G)
        
        n2v.save(DIR +'_node2vec')
       
    else:
        n2v = Node2Vec.load( fname + '.zip')
    return n2v


In [17]:
# ------------------------------------------------
# Returns idx, size of component, list of nodes
# ------------------------------------------------

def get_graph_components(graph_obj):
    component_id = 0
    components = {}
    component_size_dict = {}

    for c in nx.connected_components(graph_obj):
        components[component_id] = c
        component_size_dict[component_id] = len(c) 
        component_id += 1
        
    component_size_dict = sorted(component_size_dict.items(), key=operator.itemgetter(1), reverse=True)
    
    components_dict = []
    for i, _size in component_size_dict:
        components_dict.append((i, _size,   components[i]))
      
    return components_dict

In [18]:

# -----------------------------------------------------------
# Get ordered list of elements that are approx most dissimilar
# ------------------------------------------------------------
def query_ordered_dissimilar(ent_id, entity_type, entity_type_list , query_model, exclude_list= None):
     
    k = int(len(entity_type_list[entity_type]) * .20)
    _dict = { e1: query_model.similarity(str(ent_id),str(e1)) for e1 in entity_type_list[entity_type] }
    sorted_dict = sorted(_dict.items() ,key=operator.itemgetter(1), reverse=True)
    
    if exclude_list is not None:
        res = [ _[0] for _ in sorted_dict[-k:] if _[0] not in exclude_list]
    else:
        res = [ _[0] for _ in sorted_dict[-k:] if _[0] ]
    return res
# Example : query_ordered_dissimilar(75, 'Carrier', entity_type_list , query_model)
# -------------------------------------------------------------

def query_ordered_similar(ent_id, entity_type, entity_type_list , query_model, exclude_list= None, include_list = None):
    k = int(len(entity_type_list[entity_type]) * .25)
    _dict = { e1: query_model.similarity(str(ent_id),str(e1)) for e1 in entity_type_list[entity_type] }
    sorted_dict = sorted(_dict.items() ,key=operator.itemgetter(1), reverse=True)
    
    if exclude_list is not None:
        res = [ _[0] for _ in sorted_dict[:k] if _[0] not in exclude_list]
    else:
        res = [ _[0] for _ in sorted_dict[:k]  ]
   
    if include_list is not None:
        res = [ _ for _ in res if _ in include_list]
    else:
        res = [ _ for _ in res ]
        
    return res
# Example : query_ordered_similar(75, 'Carrier', entity_type_list , query_model)
# -------------------------------------------------------------

In [19]:
# --------------------------------------------------
# Return non-cooccurring entities
# Example : get_nonCoOcc(45, 'Carrier', 'HSCode'))
# Return entity of type HSCode which does not co-occur with Carrier:45
# --------------------------------------------------

def get_nonCoOcc(ent_id, e_type, target_e_type):
    global main_df
    global entity_type_list
    tmp = set(main_df[[e_type, target_e_type]].loc[main_df[e_type]==ent_id][target_e_type].values.tolist())
    return list(set(entity_type_list[target_e_type]).difference(tmp))


In [20]:
n2v = get_n2v_model(complete_G.copy())
# Save model to gensim.KeyedVector format
n2v.save_vectors("{}_n2v_model.bin".format(DIR))
query_model = KeyedVectors.load_word2vec_format("{}_n2v_model.bin".format(DIR))

In [21]:
# =======================================
company_cols = ['ConsigneePanjivaID', 'ShipperPanjivaID']

df = target_df.copy()
print('>> ', len(df))
attributes = [ _ for _ in list(df.columns) if _ not in id_col]
df = df.drop_duplicates(subset = attributes)
df_subset = df[company_cols].groupby(
    company_cols).size().reset_index(
    name='count'
).sort_values(by='count', ascending=False)


# --------------------
# Create a bipartite graph
# --------------------
B = nx.Graph()
B.add_nodes_from(set(df_subset['ConsigneePanjivaID'].values), bipartite=0)
B.add_nodes_from(set(df_subset['ShipperPanjivaID'].values), bipartite=1)
edges = []

for i, j, k in zip(df_subset['ConsigneePanjivaID'].values,
                   df_subset['ShipperPanjivaID'].values,
                   df_subset['count'].values):
    edges.append((i, j, {'weight': k}))

B.add_edges_from(edges)
print(' Is the bipartite graph of Comapnies connected ? ', nx.is_connected(B))

>>  68257
 Is the bipartite graph of Comapnies connected ?  False


In [22]:
B0, _ = get_largest_component_subgraph(B)
print('Nodes in the largest component', B0.number_of_nodes())

Nodes in the largest component 10130


In [23]:
bt_sorted = betweenness_centrality_parallel(B0)
bt_sorted = sorted(bt_sorted.items(), key=operator.itemgetter(1), reverse=True)

centrality_d = nx.degree_centrality(B0)
centrality_d = sorted(centrality_d.items(), key=operator.itemgetter(1), reverse=True)

In [24]:
# -----------------------------
# Top comapnies in terms of Between Ness centrality and Degree Centrality
# Set K = 50
# -----------------------------
K = 100
top_K_companies = set([_[0] for _ in  bt_sorted[:K] + centrality_d[:K]])
top_K_companies = list(set(top_K_companies))
tmp1 = df.loc[(df['ConsigneePanjivaID'].isin(top_K_companies)) | (df['ShipperPanjivaID'].isin(top_K_companies))].copy()

# The set of HS codes that are mostly transacted
popular_hscodes = set(tmp1['HSCode'])
hscode_degree_dict = {node_hsc : complete_G.degree(node_hsc) for node_hsc in popular_hscodes}
K = int(len(hscode_degree_dict)*0.5)
top_K_hscodes = [_[0] for _ in sorted(hscode_degree_dict.items() , key = operator.itemgetter(1), reverse=True)][:K]

In [25]:
# =====================================
# To get community/clusters of nodes ,
# We remove some highly visible nodes
# =====================================
rmv = []
K = 1000
for sorted_dict in [ bt_sorted, centrality_d]:
    rmv.extend( [ _[0] for _ in sorted_dict[:K] ])
rmv = list(set(rmv))

In [26]:
B1 = nx.Graph(B0)
print(B1.number_of_edges(), B1.number_of_nodes())

B1.remove_nodes_from(rmv)
print(B1.number_of_edges(), B1.number_of_nodes())

22382 10130
5942 8738


In [27]:
max_comp, _components = get_largest_component_subgraph(B1)
g_components = get_graph_components(B1)

In [28]:
# set_PoL =  set(df['PortOfLading'])
# PoL_distances = {}
# for h in set_PoL:
#     _dict = {_h:len(nx.shortest_path(complete_G, h,_h))-2 for _h in set_PoL if _h != h }
#     PoL_distances[h] = sorted(_dict.items(), key = operator.itemgetter(1), reverse=True)
domain_dims = get_domain_dims()
domain_dims

{'Carrier': 589,
 'ConsigneePanjivaID': 5817,
 'HSCode': 125,
 'PortOfLading': 243,
 'PortOfUnlading': 67,
 'ShipmentDestination': 126,
 'ShipmentOrigin': 115,
 'ShipperPanjivaID': 7271}

In [29]:
required_length = int(len(df_test)*config_ANOM_PERC/100)


In [30]:
positive_candidate = None


# Leave out the largest community
for i in range(1,10):
    g = B1.subgraph(g_components[i][-1])
    # what are the entity ids ?
    
    # Omit nodes which have degree 1
    _nodes_ = [ _ for _ in g.nodes() if g.degree(_) > 1]
    tmp_df = df.loc[(df['ConsigneePanjivaID'].isin(_nodes_)) & (df['ShipperPanjivaID'].isin(_nodes_))].copy()
  
    _ratio = len(tmp_df) * config_SIMTYPE / required_length
    print(len(tmp_df), _ratio)
    if _ratio <= 1.3 and _ratio >= 0.75 :
        print('Valid candidate found')
        tmp_df = tmp_df.sample(n = int(required_length/config_SIMTYPE), replace=True) 
        print(
            ' Length of the candidate df >>', 
            tmp_df.shape[0], 
            '# Consignee & Shipper ', 
            len(set(tmp_df['ConsigneePanjivaID'])), 
            len(set(tmp_df['ShipperPanjivaID']))
        )
        positive_candidate = tmp_df.copy().reset_index(drop=True)
        candidate_df = tmp_df.copy()
        break
        

444 5.103448275862069
132 1.5172413793103448
111 1.2758620689655173
Valid candidate found
 Length of the candidate df >> 87 # Consignee & Shipper  26 26


In [31]:
# g = B.subgraph(g_components[1][-1])
# print(nx.info(g))
# # what are the entity ids ?
# list_e = list(g.nodes())
# tmp_df = df.loc[(df['ConsigneePanjivaID'].isin(list_e)) & (df['ShipperPanjivaID'].isin(list_e))].copy()

In [32]:
# set(positive_candidate['HSCode'])
# query_ordered_dissimilar(ent_id, entity_type, entity_type_list , query_model, exclude_list= None)

In [33]:
def generate_aux(
    cand_df, 
    idx, 
    num_samples, 
    perturb_extra= True
):
    global df
    global query_model
    global entity_type_list
    global id_col
    global domain_dims
    global config_SIMTYPE # Ensue this is an even number 
    print(' IDX', idx)
    # Choose the anchor point
    a = np.random.uniform()
    domain_combinations = { 1 : { 
            'company_anchor' : 'ConsigneePanjivaID',
            'domain_set' : {
            1 : ('HSCode',  'ShipmentOrigin'),
            2 : ('Carrier', 'PortOfLading'),
            3 : ('HSCode',  'Carrier')}},
        2 : { 
            'company_anchor' : 'ShipperPanjivaID',
            'domain_set' : {
            1 : ( 'HSCode', 'ShipmentDestination'),
            2 : ( 'Carrier', 'PortOfUnlading'),
            3 : ( 'HSCode',  'Carrier'),
            }
        }
    } 
    req_count = num_samples 
    c_idx = [1,2]
    ds_idx = [1,2,3]

    np.random.shuffle(c_idx)
    np.random.shuffle(ds_idx)
    _options  =[]
    for i in c_idx:
        for j in ds_idx:
            _options.append((i,j))
    for i_j in _options:
        i = i_j[0]
        j = i_j[1]
        
        company_anchor = domain_combinations[i]['company_anchor']
        d_set = domain_combinations[i]['domain_set'][j]
      
        print(d_set)
        if perturb_extra:
            perturb_columns = [ _ for _ in domain_dims.keys() if _ not in ['ConsigneePanjivaID','ShipperPanjivaID'] + list(d_set) ]

        # Company entity
        company_ent = cand_df.loc[idx, company_anchor]
        # entity that does not co-occur with company entity
        a1 = get_nonCoOcc(ent_id=company_ent, e_type=company_anchor, target_e_type=d_set[0])
        domain_0 = d_set[0]
        domain_1 = d_set[1]
        # entity that is most dissimilar to company entity : domain_0
        a2 = query_ordered_dissimilar(
            ent_id = company_ent, 
            entity_type = domain_0, 
            entity_type_list = entity_type_list,
            query_model=query_model,
            exclude_list= None
        )
        
        a3 = list(set(a1).intersection(set(a2)))
        
        # entity that is most dissimilar to company entity : domain_1
        b1 = get_nonCoOcc(ent_id = company_ent, e_type = company_anchor, target_e_type=domain_1)
        b2 =  query_ordered_dissimilar(
                ent_id = company_ent, 
                entity_type = domain_1, 
                entity_type_list = entity_type_list,
                query_model=query_model,
                exclude_list= None
            )
        b3 = list(set(b1).intersection(set(b2)))
        if a3 is None or b3 is None or len(a3) == 0 or len(b3)==0  : 
            continue
        e1_0 = np.random.choice(a3, size=1, replace=False)[0]
        e2_0 = np.random.choice(b3, size=1, replace=False)[0]
        
        e1_ = query_ordered_similar(
            ent_id = e1_0, 
            entity_type = d_set[0], 
            entity_type_list = entity_type_list,
            query_model=query_model,
            exclude_list= [e1_0],
            include_list = list(a3)
        )

        e1 = [e1_0] + e1_[:req_count-1]
        e2_ = query_ordered_similar(
            ent_id = e2_0, 
            entity_type = d_set[1], 
            entity_type_list = entity_type_list,
            query_model=query_model,
            exclude_list= [e2_0],
            include_list = list(b3)
        )
        e2 = [e2_0] + e2_[:req_count-1]
        print( ' >>>' ,len(e1), len(e2))
        try:
            list_ent_1 = []
            list_ent_2 = []

            for _i in range(req_count):
                if _i%2 == 0:
                    list_ent_1.extend([e1[_i],e1[_i]])
                    list_ent_2.extend([e2[_i]] )
                if _i%2 == 1:
                    list_ent_1.extend([e1[_i]])
                    list_ent_2.extend([e2[_i],e2[_i]] )
                if  len(list_ent_1) >= req_count  or len(list_ent_2)>=req_count:
                    break
            
            res = []
            cause = {}
            print(list_ent_1, list_ent_2)
            for i in range(req_count):
                row_copy = cand_df.loc[idx].copy()
                row_copy[id_col] = int(str(row_copy[id_col]) + '00{}'.format(i+1))
                row_copy[domain_0]= list_ent_1[i]
                row_copy[domain_1]= list_ent_2[i]
                _p = np.random.choice(perturb_columns,size=1)[0]
            
                row_copy[_p] = np.random.choice(entity_type_list[_p],size=1)[0]
                res.append(row_copy)
                cause[str(row_copy[id_col])] = [(company_anchor,domain_0),(company_anchor,domain_1)]
            res = pd.concat(res,axis=1).transpose().reset_index(drop=True)
            return (res, cause)
        except:
            continue
        
    return (None, None)
        

In [34]:
# =========================
# Positive samples
# =========================

candidate_df = candidate_df.reset_index(drop=True)

In [35]:
num_samples = config_SIMTYPE
results = Parallel(
    n_jobs = mp.cpu_count()
)(delayed(generate_aux)( candidate_df, idx, num_samples, ) for idx in range(candidate_df.shape[0]))

In [36]:
df_results = [] 
causes = {}
for r in results:
    if r[0] is not None:
        df_results.append(r[0])
        causes.update(r[1])
        
positive_anomalies = pd.concat(df_results).reset_index(drop=True)

In [37]:
# ---------------------------
# Save the causes
# ---------------------------
pos_anom_causes_file = os.path.join(Anomaly_Output_path,'pos_anomalies_explantions.json')
print(pos_anom_causes_file)
with open(pos_anom_causes_file, "w") as fh:  
    json.dump(causes, fh) 


./../generated_data_v1/generated_anomalies/us_import2/pos_anomalies_explantions.json


In [38]:
# ==========================
# Negative anomalies
# ==========================
# Consider the largest component
g = B1.subgraph(g_components[0][-1])
_nodes_ = [ _ for _ in g.nodes() if g.degree(_) > 1]
tmp_df = df.loc[(df['ConsigneePanjivaID'].isin(_nodes_)) & (df['ShipperPanjivaID'].isin(_nodes_))].copy()

In [39]:
negative_candidates = tmp_df.sample(positive_anomalies.shape[0],replace=False).reset_index(drop=True)

In [40]:
def generate_perturbed_aux(cand_df, idx, num_pert = 3):
    global entity_type_list
    global id_col
    global company_attributes
    row_copy = cand_df.loc[idx].copy()
    perturb_domains = [ _ for _ in list(entity_type_list.keys()) if _ not in company_attributes ]
    perturb_domains = np.random.choice(perturb_domains, num_pert)
    
    for i in range(num_pert):
        _dom = perturb_domains[i]
        row_copy[_dom] = np.random.choice(entity_type_list[_dom],size=1)[0]
    row_copy[id_col] = int(str(row_copy[id_col]) + '00{}'.format(1))
    res = pd.concat([row_copy],axis=1).transpose().reset_index(drop=True)
    return res


In [41]:
res = Parallel(
    n_jobs = mp.cpu_count()
    )(delayed(generate_perturbed_aux)( 
        negative_candidates, idx, 
    ) for idx in range(negative_candidates.shape[0])
 )


In [42]:
negative_anomalies = pd.concat(res).reset_index(drop=True)

In [43]:
negative_anomalies

,PanjivaRecordID,Carrier,ConsigneePanjivaID,HSCode,PortOfLading,PortOfUnlading,ShipmentDestination,ShipmentOrigin,ShipperPanjivaID
0,112331093001,39,6116,6443,6743,6818,6947,6970,9590
1,112328996001,499,1987,6521,6699,6818,6848,6987,10745
2,113233858001,378,5414,6441,6626,6781,6865,7021,9232
3,110077911001,184,1158,6514,6582,6780,6890,6991,12980
4,109992843001,282,5456,6520,6600,6840,6943,7007,11648
...,...,...,...,...,...,...,...,...,...
343,110703776001,514,2951,6520,6641,6837,6959,6987,9446
344,110174383001,56,4296,6512,6671,6837,6959,6987,8609
345,110977940001,40,5426,6509,6582,6800,6923,6987,11573
346,113172348001,279,4799,6509,6771,6780,6959,7019,9073


In [44]:
# ==========================
# Save the data frames
# ==========================
neg_anom_f_path = os.path.join(Anomaly_Output_path,'neg_anomalies.csv')
pos_anom_f_path = os.path.join(Anomaly_Output_path,'pos_anomalies.csv')
negative_anomalies.to_csv(neg_anom_f_path,index=None)
positive_anomalies.to_csv(pos_anom_f_path,index=None)